In [ ]:
import cv2
import tensorflow as tf
from tensorflow import keras  
import numpy as np
from tensorflow.keras.models import load_model
import h5py


model=load_model(r"C:\Users\frigu\Downloads\Orange\IA\my_model.h5")
results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}
rect_size = 4
cap = cv2.VideoCapture(0) 
#~/OpenCV/opencv/data/haarcascades/haarcascade_frontalface_default.xml
haarcascade =cv2.CascadeClassifier (r'C:\Users\frigu\AppData\Local\Programs\Python\Python310\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 
    
#~/OpenCV/opencv/data/haarcascades/haarcascade_frontalface_default.xml
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    cv2.imshow('frame', rerect_size)
    if cv2.waitKey(1) == ord('q'):
        break
    
    faces = haarcascade.detectMultiScale(rerect_size,scaleFactor=1.05,minNeighbors=5)#
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
import tensorflow as tf

model=load_model(r"C:\Users\frigu\Downloads\Orange\IA\my_model.h5")
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
import cv2
import tensorflow as tf
from tensorflow import keras  
import numpy as np
from tensorflow.keras.models import load_model
import h5py


model=load_model(r"C:\Downloads\Orange\IA\my_model.h5")
results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}
rect_size = 4
im = cv2.imread(r'E:\image2.jpg') 
#~/OpenCV/opencv/data/haarcascades/haarcascade_frontalface_default.xml
haarcascade =cv2.CascadeClassifier (r'C:\AppData\Local\Programs\Python\Python310\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')
im=cv2.flip(im,1,1) 
cv2.imshow('LIVE',im)
cv2.waitKey(0)
    
#~/OpenCV/opencv/data/haarcascades/haarcascade_frontalface_default.xml
rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))

    
faces = haarcascade.detectMultiScale(rerect_size,scaleFactor=1.05,minNeighbors=5)#

for f in faces:
    (x, y, w, h) = [v * rect_size for v in f] 
        
    face_img = im[y:y+h, x:x+w]
    rerect_sized=cv2.resize(face_img,(150,150))
    normalized=rerect_sized/255.0
    reshaped=np.reshape(normalized,(1,150,150,3))
    reshaped = np.vstack([reshaped])
    result=model.predict(reshaped)
        
    label=np.argmax(result,axis=1)[0]
    print(f"label : {result}")
      
    cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
    cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
    cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imwrite(r'E:\image_ma8ir mask.jpg',im)
    cv2.imshow('LIVE',im)
    cv2.waitKey(0)

cv2.destroyAllWindows()

# ai.py

In [ ]:
import cv2
import tflite_runtime.interpreter as tf
import numpy as np
import h5py

def Predect():
    # create object
    #model=tf.keras.Model()
    interpreter = tf.Interpreter(r'/home/pi/Desktop/model.tflite')

    interpreter.allocate_tensors()
    output_details = interpreter.get_output_details()
    #model=load_model(r"C:\Users\frigu\Downloads\Orange\IA\my_model.h5")
    results={0:'without mask',1:'mask'}
    GR_dict={0:(0,0,255),1:(0,255,0)}
    rect_size = 4
    im = cv2.imread('/home/pi/Desktop/image.jpg')
    #~/OpenCV/opencv/data/haarcascades/haarcascade_frontalface_default.xml
    haarcascade=cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml") #Note the change
    im=cv2.flip(im,1,1)     
    #~/OpenCV/opencv/data/haarcascades/haarcascade_frontalface_default.xml
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))

    faces = haarcascade.detectMultiScale(rerect_size)#
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
          
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        interpreter.invoke()
        
        #result=interpreter.predict(reshaped)
        output_data = interpreter.get_tensor(output_details[0]['index'])
        print(type(output_data))
        print(output_data[0][0])
        print(", the output is {}".format(output_data))    
        label=np.argmax(output_data,axis=1)[0]
        print(label) 
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        print(GR_dict[label])
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        cv2.imshow('frame',   im)
        cv2.waitKey(0)
        return im

    cv2.destroyAllWindows()
    
    



# Script.py

In [ ]:
import mysql.connector
import datetime

def DBConnection():
    db = mysql.connector.connect(
        host=".....",
        username="......",
        password="......",
        db="...."
    )
    return db

def get_all_user_ids(curseur, db):
    try:
        curseur.execute(f"select id_employee from user")
        users = curseur.fetchall()    
        users = [users[l][0] for l in range(len(users)) ] 
        users = [user.split(":") for user in users]
        users = [[int(l) for l in user] for user in users]
    except:
        print('No users ids!!!')
    return users

def get_all_user_ids(curseur, db):
    try:
        curseur.execute(f"select DISTINCT id_employee from user")
        users = curseur.fetchall()    
        users = [users[l][0] for l in range(len(users)) ] 
        users = [user.split(":") for user in users]
        users = [[int(l) for l in user] for user in users]
    except:
        print('No users ids!!!')
    return users

def add_snapshot(user_id, verdict, name="Goerge", email="goerge@gmail.com"):
    now = datetime.datetime.now()
    # 2010-12-03 16:01:06
    now = now.strftime("%Y-%m-%d %H:%M:%S")
    try:
        curseur.execute(f"INSERT INTO `entrance_snapshot`(`id_employee`, `name`, `email`, `time`, `verdict`) VALUES ('{user_id}','{name}','{email}','{now}',{verdict});")
        db.commit()
    except:
        print('Error')
    # print((f"INSERT INTO `entrance_snapshot`(`id_employee`, `name`, `email`, `time`, `verdict`) VALUES ({user_id},{name},{email},{now},{verdict});"))
    db.commit()



# Main
db = DBConnection()
curseur = db.cursor()
# users_ids =  get_all_user_ids(curseur, db)
# print(users_ids)
# print(type(users_ids))

#add_snapshot("185:226:95:179:183", False)

db.close()


# RFID_camera.py

In [ ]:
import RPi.GPIO as GPIO #Importe la bibliothèque pour contrôler les GPIOs
from pirc522 import RFID

from picamera import PiCamera
from time import sleep
import cv2
from Script import *
from ai import *


GPIO.setmode(GPIO.BOARD) #Définit le mode de numérotation (Board)
GPIO.setwarnings(False) #On désactive les messages d'alerte
camera = PiCamera()

LED_RED = 3 #Définit le numéro du port GPIO qui alimente la led rouge
LED_GREEN = 5 #Définit le numéro du port GPIO qui alimente la led verte

test=False
rc522 = RFID() #On instancie la lib

def turn_led_on (led) :
    GPIO.setup(led, GPIO.OUT) #Active le contrôle du GPIO
    GPIO.output(led, GPIO.HIGH) #Allume la led

#Définit la fonction permettant d'éteindre une led
def turn_led_off (led) :
    GPIO.setup(led, GPIO.OUT) #Active le contrôle du GPIO
    GPIO.output(led, GPIO.LOW) #Eteind la led

#Définit la fonction permettant d'allumer la rouge et éteindre la verte
def turn_red_on () :
    turn_led_off(LED_GREEN) #Eteind la led verte
    turn_led_on(LED_RED) #Allume la led rouge

#Définit la fonction permettant d'allumer la verte et éteindre la rouge
def turn_green_on () :
    turn_led_off(LED_RED) #Eteind la led rouge
    turn_led_on(LED_GREEN) #Allume la led verte

#Définit la fonction permettant d'allumer la rouge et éteindre la verte
def Take_Picture() :
    camera.start_preview()
    sleep(3)
    camera.capture('/home/pi/Desktop/image.jpg')
    camera.stop_preview()
    im='/home/pi/Desktop/image.jpg'
    return im



print('En attente d\'un badge (pour quitter, Ctrl + c): ') #On affiche un message demandant à l'utilisateur de passer son badge

#On va faire une boucle infinie pour lire en boucle
while True :
    rc522.wait_for_tag() #On attnd qu'une puce RFID passe à portée
    (error, tag_type) = rc522.request() #Quand une puce a été lue, on récupère ses infos

    if not error : #Si on a pas d'erreur
        (error, uid) = rc522.anticoll() #On nettoie les possibles collisions, ça arrive si plusieurs cartes passent en même temps
        if not error : #Si on a réussi à nettoyer
            db = DBConnection()
            curseur = db.cursor()
            users_ids=get_all_user_ids(curseur, db)
            
            print(f"Base de données : {users_ids}")
            for RFID_UID in users_ids :
                if RFID_UID == uid :
                    print('Badge {} autorisé !'.format(uid))
                    turn_green_on()
                    im=Take_Picture()
                    test=True
                    GPIO.output(LED_GREEN, GPIO.LOW) #Eteind la led
                    #im=Predect(im)
                    cv2.imshow('frame',im)
                    cv2.waitKey(0)
                    break
            if not test:
                print('Badge {} interdit !'.format(uid))
                turn_red_on()

        test=False
        sleep(2) #On attend 1 seconde pour ne pas lire le tag des centaines de fois en quelques milli-secondes
        GPIO.output(LED_GREEN, GPIO.LOW) #Eteind la led
        GPIO.output(LED_RED, GPIO.LOW) #Eteind la led
        # Main
        # users_ids =  get_all_user_ids(curseur, db)
        # print(users_ids)
        # print(type(users_ids))

        #add_snapshot("185:226:95:179:183", False)
        db.close()

